# Homework 3
- Madison Rusch
- Mark Minxing Zhao
- Tim Taylor
- Qayum Khan

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

def generate_treasury_cashflow_matrix(bond_maturity, coupon_rate, face_value, paid_per_year):
    cashflow_matrix = []
    for i in range(1, (bond_maturity * paid_per_year) + 1) :
        if i == bond_maturity * paid_per_year:
            cashflow_matrix.append(face_value + (face_value * (coupon_rate/paid_per_year)))
        else:
            cashflow_matrix.append(face_value * (coupon_rate/paid_per_year))
    return cashflow_matrix

def generate_swap_cashflow_matrix(length_of_swap, swap_rate, notional, paid_per_year):
    cashflow_matrix = []
    for i in range(1, (length_of_swap * paid_per_year) + 1):
        if i == length_of_swap * paid_per_year:
            cashflow_matrix.append(notional + (notional * (swap_rate/paid_per_year)))
        else:
            cashflow_matrix.append(notional * (swap_rate/paid_per_year))
    return cashflow_matrix

def generate_payment_dates(years, frequency_per_year, start_date: datetime.date):
    dates = []
    current_date = start_date
    for i in range(years*frequency_per_year):
        months = 12 / frequency_per_year
        new_date = current_date + pd.DateOffset(months=months)
        current_date = new_date
        dates.append(new_date)
    return dates

## 1.1
List the projected cashflows on May 4, 2009, exactly six months into the trade, on the first coupon and swap date.

In [38]:
# Treasury
MATURITY = 30
COUPON = 0.045
FACE_VALUE = 100
T_PAID_PER_YEAR = 2
YIELD_TO_MATURITY = 0.04193

# Swap
LENGTH = 30
SWAP_RATE = 0.04256
NOTIONAL = 100
S_PAID_PER_YEAR = 2

CURRENT_DATE = datetime.date(2008, 11, 4)

treasury_cashflows = generate_treasury_cashflow_matrix(MATURITY, COUPON, FACE_VALUE, T_PAID_PER_YEAR)
swap_cashflows = generate_swap_cashflow_matrix(LENGTH, SWAP_RATE, NOTIONAL, S_PAID_PER_YEAR)
dates = generate_payment_dates(LENGTH, T_PAID_PER_YEAR, CURRENT_DATE)

df = pd.DataFrame(index=dates, columns=['Treasury', 'Swap'])
df['Treasury'] = treasury_cashflows
df['Swap'] = swap_cashflows
display(df.iloc[0])

Treasury    2.250
Swap        2.128
Name: 2009-05-04 00:00:00, dtype: float64

We receive the treasury coupon rate as income, and pay out the fixed rate of the swap, as seen above on a 100 face value, resulting in a total cashflow of $0.12 per $100.

## 1.2
What is the duration of the T-bond? The swap?

Remember that...

- the swap can be decomposed into a fixed-rate bond and a floating-rate note
- a floating-rate note has duration equal to the time until the next reset. Thus, at initialization, it has duration equal to 0.5 years.

Is the duration for the "paying-fixed" swap positive or negative? Is it bigger or smaller in magnitude than the T-bond?

For this problem, calculate the Macauley duration and the dollar (Macauley) duration.

In [39]:
def calculate_duration(cashflow_matrix, yield_to_maturity, frequency=2):
    numerator = 0
    denominator = 0
    for i, cashflow in enumerate(cashflow_matrix):
        numerator += cashflow * (i+1) / (1 + yield_to_maturity / frequency) ** (i+1)
        denominator += cashflow / (1 + yield_to_maturity / frequency) ** (i+1)
    return numerator / denominator

def duration(coupon_rate, time_to_maturity, frequency, rate):
    """
    Calculates the time intervals of the cashflows and returns the average 
    of the the time intervals weighted by the corresponding cashflows
    """
    if coupon_rate==0:
        return time_to_maturity
    else:
        first_coupon_interval = time_to_maturity%(1/frequency)
        number_of_payments = int(np.ceil(time_to_maturity/(1/frequency)))
        time_intervals = np.linspace(first_coupon_interval, time_to_maturity, number_of_payments)
        cashflows = []
        for Ti_t in time_intervals:
            cashflows.append(((coupon_rate/frequency)/np.power(1+rate/frequency, frequency*(Ti_t))))
        cashflows[-1] += 100/np.power(1+rate/frequency, frequency*time_to_maturity)
        duration = np.average(time_intervals, weights=cashflows)
        return duration
    
def duration_2(coupon_rate, ytm, years_left, freq = 2):
    c = coupon_rate/(100*freq)
    y = ytm/(100*freq)
    n = years_left*freq
    m = freq
    macaulay_duration = ((1+y) / (m*y)) - ( (1 + y + n*(c-y)) / ((m*c* ((1+y)**n - 1)) + m*y) )
    modified_duration = macaulay_duration / (1 + y)
    return macaulay_duration, modified_duration


# treasury_duration = calculate_duration(df['Treasury'], YIELD_TO_MATURITY, T_PAID_PER_YEAR)
treasury_duration = duration_2(COUPON*100,YIELD_TO_MATURITY*100, MATURITY, T_PAID_PER_YEAR)[0]
print(f'The duration of the treasury is {treasury_duration}')

# swap_fixed_duration = calculate_duration(df['Swap'], SWAP_RATE, S_PAID_PER_YEAR)
swap_fixed_duration = duration_2(SWAP_RATE*100, SWAP_RATE*100, LENGTH, S_PAID_PER_YEAR)[0]
swap_floating_duration = 0.5 # On Nov 4, 2008
# For 'paying-fixed', we are short the fixed_duration, long the floating_duration
swap_duration = swap_floating_duration - swap_fixed_duration
print(f'The duration of the swap is {swap_duration}')

print(f'Therefore the net duration is {treasury_duration + swap_duration}')

The duration of the treasury is 17.083633069693033
The duration of the swap is -16.712744454567797
Therefore the net duration is 0.3708886151252351


Note that in the above calculation, the Swap duration comes out to be negative, with a slightly smaller magnitude than the duration of the Treasury Bond. We also made the assumption that the YTM of the of the Fixed portion of the Swap was equivalent to the swap rate.

In [40]:
def calculate_modified_duration(duration, yield_to_maturity, frequency=2):
    return duration/(1+(yield_to_maturity/frequency))

def calculate_dollar_duration(price, duration, yield_to_maturity, frequency=2):
    md = calculate_modified_duration(duration, yield_to_maturity, frequency)
    return price * md

# treasury_modified_duration = duration_2(COUPON*100,YIELD_TO_MATURITY*100, MATURITY, T_PAID_PER_YEAR)[1]
treasury_dollar_duration = calculate_dollar_duration(FACE_VALUE, treasury_duration, YIELD_TO_MATURITY, T_PAID_PER_YEAR)
print(f'The dollar duration of the treasury is {treasury_dollar_duration}')

# swap_modified_duration = duration_2(SWAP_RATE*100, SWAP_RATE*100, LENGTH, S_PAID_PER_YEAR)[0]
swap_dollar_duration = calculate_dollar_duration(NOTIONAL, swap_duration, SWAP_RATE, S_PAID_PER_YEAR)
print(f'The dollar duration of the swap is {swap_dollar_duration}')


The dollar duration of the treasury is 1673.2829303348335
The dollar duration of the swap is -1636.4507730071869


## 1.3
What hedge ratio should be used to balance the notional size of the Treasury bond with the notional size of the swap, such that it is a duration-neutral position?

Specifically, if the trader enters the swap paying fixed on $500 million notional, how large of a position should they take in the Treasury bond?

In [41]:
# For hedging, we need the ratio of durations
hedge_ratio = swap_dollar_duration / treasury_dollar_duration
print(f'The hedge ratio is {hedge_ratio}')

treasury_hedge = round(-500_000_000 * hedge_ratio, 2)
print(f'In order to hedge a $500 million notional on the swap, the trader needs to go long ${"{:,}".format(treasury_hedge)} in Treasury bonds')

The hedge ratio is -0.9779880875732855
In order to hedge a $500 million notional on the swap, the trader needs to go long $488,994,043.79 in Treasury bonds


## 1.4
Suppose it is May 4, 2009, exactly six months after putting the trade on.

The spread is at -28 bps due to...

- The YTM on a new 30-year bond has risen to 4.36%
- The swap rate on a new 30-year swap has dropped to 4.08%

Explain conceptually how this movement impacts the components of the trade.

With the higher YTM on the new bond, the price of the bonds have fallen, and our position has lost value. In terms of the swap, the lowered swap rate means we are overpaying on the fixed portion of our existing swap, and we have lost value on the swap as well. This conceptually makes sense because we bet on the spread widening and instead it shrank and even flipped, meaning we lost money on both sides of the trade.

## 1.5
Calculate the value of the position on May 4, 2009, immediately after the first coupon and swap payments and swap reset.

Calculate the revised price of the Treasury bond by assuming you can apply the (May 4) 30-year YTM as a discount rate to the 29.5 year bond. (We are just using this for a rough approximation. You know that good pricing would require a discount curve, but let's not get bogged down with that here.)

Calculate the value of the swap by decomposing it into a fixed-rate bond and a floating-rate bond.

- The 29.5 year fixed-rate leg is priced using the (May 4) 30-year swap rate as a discount rate.
- The floating-rate leg is priced at par given that floating-rate notes are par immediately after resets.

In [47]:
""" Get bond price from YTM """
def bond_price(face_value, maturity_in_years, ytm, coupon, frequency=2):
    freq = float(frequency)
    periods = maturity_in_years*freq
    numerator = coupon*face_value/freq
    dt = [(i+1)/freq for i in range(int(periods))]
    price = sum([numerator/(1+ytm/freq)**(freq*t) for t in dt]) + \
            face_value/(1+ytm/freq)**(freq*maturity_in_years)
    return price

years_left = 29.5
new_ytm = 0.0436
new_treasury_price = bond_price(FACE_VALUE, years_left, new_ytm, COUPON, T_PAID_PER_YEAR)
print(f'The revised price of the Treasury bond is ${round(new_treasury_price, 2)}')

new_swap_rate = 0.0408
swap_fixed_value = bond_price(NOTIONAL, years_left, new_swap_rate, SWAP_RATE, S_PAID_PER_YEAR)
swap_floating_value = NOTIONAL
print(f'The new value of the fixed leg of the swap is ${round(swap_fixed_value, 2)}, and the new value of the floating leg is ${round(swap_floating_value, 2)}')
print(f'Thus the total net change in value of the Swap (given that we\'re short) is ${round(swap_floating_value - swap_fixed_value, 2)} per $100 face value')

The revised price of the Treasury bond is $102.31
The new value of the fixed leg of the swap is $103.0, and the new value of the floating leg is $100
Thus the total net change in value of the Swap (given that we're short) is $-3.0 per $100 face value


Given the rising YTM, and given the original price of the bond was $105, it is not a surprise to see the price drop to $102.31.

Similarly for the Swap, since the swap rate has dropped, the fixed leg of the swap has gone up in value, and we're short, so overall our position has lost value.

## 1.6
Accounting for the change in value of the positions, as well as the 6-month cashflows paid on May 4,

- What is the net profit and loss (pnl) of the position?
- What is the return on the equity capital, considering that there was a 2% haircut (equity contribution) on the size of the initial treasury bond position.

In [48]:
dirty_price = 105
treasury_cashflow_1 = treasury_hedge * COUPON
treasury_pnl = (treasury_hedge / dirty_price) * (new_treasury_price - dirty_price)

swap_notional = 500_000_000
swap_payout_1 = -swap_notional * SWAP_RATE
swap_pnl = (swap_notional / NOTIONAL) * (swap_floating_value - swap_fixed_value)
pnl = treasury_cashflow_1 + treasury_pnl + swap_payout_1 + swap_pnl
print(f'The total PNL on the position is ${"{:,}".format(round(pnl, 2))}')

The total PNL on the position is $-26,813,053.63


In [50]:
equity = treasury_hedge * 0.02
return_on_equity = (pnl/equity) * 100

print(f'The return on equity capital is {round(return_on_equity, 3)}%')

The return on equity capital is -274.165%
